In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP project/cleaned_v2.xlsx", index_col=0)

labels = pd.read_excel(r"/content/drive/MyDrive/Colab Notebooks/NLP project/processed/claims_russia.xlsx", index_col=0)

labels = list(labels.index.to_numpy())
labels

In [5]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"DEVICE: {device} \n---------------------------------------------------------\n\n")

model_name = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

premise = "I first thought that I liked the movie, but upon second thought it was actually disappointing."
hypothesis = "The movie was not good."

input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
DEVICE: cuda 
---------------------------------------------------------


{'entailment': 83.6, 'neutral': 16.2, 'contradiction': 0.3}


In [6]:
article = list(df.cleaned_content.loc[df.uid == 'russia_260'].to_numpy())[0]
hypothesis = "Russia is enlisting soldiers with chronic health conditions"

In [7]:
articles = dict(zip(df.uid.loc[df.topic == 'russia'], df.cleaned_content.loc[df.topic == 'russia']))



In [31]:
import operator
import os.path

file_path = "/content/drive/MyDrive/Colab Notebooks/NLP project/generated claim ver/russia_claims.xlsx"

#if os.path.isfile(file_path):
#    out = pd.read_excel(file_path, index_col=0)
#else:
out = pd.DataFrame(columns=['claim', 'supporting', 'rejecting'])

for claim in labels:
    new_row = {'claim' : claim, 'supporting' : [], 'rejecting' : []}
    out = out.append(new_row, ignore_index=True)
    print(claim)
    for uid, article in articles.items():
        premise = article
        hypothesis = claim

        input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
        output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
        prediction = torch.softmax(output["logits"][0], -1).tolist()
        label_names = ["entailment", "neutral", "contradiction"]
        prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
        print(uid)
        print(prediction)
        if max(prediction, key=prediction.get) == 'entailment' and prediction[max(prediction.items(),key=operator.itemgetter(1))[0]] > 80.0:
            results = out.supporting.loc[out['claim'] == claim].item()
            results.append(uid)
        elif max(prediction, key=prediction.get) == 'contradiction' and prediction[max(prediction.items(),key=operator.itemgetter(1))[0]] > 80.0:
            results = out.rejecting.loc[out['claim'] == claim].item()
            results.append(uid)
    out.to_excel(file_path)

Streaming output truncated to the last 5000 lines.
{'entailment': 1.5, 'neutral': 97.3, 'contradiction': 1.2}
russia_136
{'entailment': 16.9, 'neutral': 31.4, 'contradiction': 51.8}
russia_137
{'entailment': 1.4, 'neutral': 94.7, 'contradiction': 3.9}
russia_138
{'entailment': 0.8, 'neutral': 95.9, 'contradiction': 3.2}
russia_139
{'entailment': 41.7, 'neutral': 43.5, 'contradiction': 14.8}
russia_140
{'entailment': 48.0, 'neutral': 26.1, 'contradiction': 26.0}
russia_141
{'entailment': 4.7, 'neutral': 92.9, 'contradiction': 2.4}
russia_142
{'entailment': 12.6, 'neutral': 61.1, 'contradiction': 26.3}
russia_143
{'entailment': 2.3, 'neutral': 96.8, 'contradiction': 0.9}
russia_144
{'entailment': 3.0, 'neutral': 85.0, 'contradiction': 12.0}
russia_145
{'entailment': 25.4, 'neutral': 65.6, 'contradiction': 9.0}
russia_146
{'entailment': 8.4, 'neutral': 89.8, 'contradiction': 1.9}
russia_147
{'entailment': 46.0, 'neutral': 27.7, 'contradiction': 26.3}
russia_148
{'entailment': 48.5, 'neutr

In [23]:
out

,claim,supporting,rejecting
0,Russia interfered in the US election,[],[]
1,Russia threatens to use nuclear weapons,[],[]


In [ ]:
import operator
prediction[max(prediction.items(),key=operator.itemgetter(1))[0]]

43.4

In [30]:
out.supporting.loc[out['claim'] == claim].item()

[]

In [ ]:
tuple(tup + (uid,))

TypeError: can only concatenate str (not "tuple") to str

In [ ]:
tup = out.rejecting.loc[out['claim'] == claim][0]
t = ()
t = (tup,) + (uid,)

In [ ]:
t

((), 'russia_9')